In [1]:
import cv2
import numpy as np
from PIL import Image
from keras.models import load_model
import skfuzzy as fuzz

# Load the trained model
model = load_model("best_model.h5")

# Load the face cascade classifier
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Fuzzy sets for emotion
emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

# Define fuzzy sets for each emotion
fuzzy_sets = {
    'angry': fuzz.trimf(np.arange(0, 1.01, 0.01), [0, 0.4, 0.8]),
    'disgust': fuzz.trimf(np.arange(0, 1.01, 0.01), [0.2, 0.6, 1]),
    'fear': fuzz.trimf(np.arange(0, 1.01, 0.01), [0.4, 0.7, 1]),
    'happy': fuzz.trimf(np.arange(0, 1.01, 0.01), [0.6, 0.9, 1]),
    'sad': fuzz.trimf(np.arange(0, 1.01, 0.01), [0, 0.3, 0.6]),
    'surprise': fuzz.trimf(np.arange(0, 1.01, 0.01), [0.4, 0.7, 1]),
    'neutral': fuzz.trimf(np.arange(0, 1.01, 0.01), [0.2, 0.6, 1])
}

# Define fuzzy rules for emotion detection
rules = [
    (fuzzy_sets['angry'], 'angry'),
    (fuzzy_sets['disgust'], 'disgust'),
    (fuzzy_sets['fear'], 'fear'),
    (fuzzy_sets['happy'], 'happy'),
    (fuzzy_sets['sad'], 'sad'),
    (fuzzy_sets['surprise'], 'surprise'),
    (fuzzy_sets['neutral'], 'neutral')
]

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, test_img = cap.read()  # Capture frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

    # Detect faces in the image
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)

    for (x, y, w, h) in faces_detected:
        cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=7)
        roi_gray = gray_img[y:y + w, x:x + h]  # Cropping region of interest (face area) from the image
        roi_gray = cv2.resize(roi_gray, (224, 224))
        img = Image.fromarray(roi_gray)
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0

        # Make predictions using the loaded model
        predictions = model.predict(img_array)

        # Calculate the degree of membership for each emotion using fuzzy logic
        membership = {}
        for i in range(len(emotions)):
            membership[emotions[i]] = fuzz.interp_membership(np.arange(0, 1.01, 0.01), fuzzy_sets[emotions[i]], predictions[0][i])

        # Use fuzzy rules to determine the dominant emotion
        dominant_emotion = 'unknown'
        max_membership = 0
        for rule in rules:
            if membership[rule[1]] > max_membership:
                max_membership = membership[rule[1]]
                dominant_emotion = rule[1]

        cv2.putText(test_img, dominant_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial Emotion Analysis', resized_img)

    if cv2.waitKey(10) == ord('q'):  # Wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 45ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 33ms/step
